In [11]:
!pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [12]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch


In [13]:
import pandas as pd

# Load the dataset (replace 'your_file.csv' with the uploaded file name)
df = pd.read_csv("spam_or_not_spam.csv")

# Show the first few rows to understand the dataset
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'spam_or_not_spam.csv'

In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Save the split datasets as CSV files
train_df.to_csv("train.csv", index=False)
test_df.to_csv("test.csv", index=False)


In [ ]:
from datasets import load_dataset

# Load the dataset files (train and test CSV files)
dataset = load_dataset("csv", data_files={"train": "train.csv", "test": "test.csv"})

# Show the dataset to ensure it's loaded correctly
print(dataset)


In [ ]:
print(dataset["train"].column_names)

In [ ]:
# Convert the 'text' column to string type if needed
dataset = dataset.map(lambda x: {"text": [str(t) for t in x["text"]]}, batched=True)

In [ ]:
from transformers import RobertaTokenizer

# Load the RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Tokenization function
def tokenize_function(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Remove the "text" column as it's no longer needed
tokenized_dataset = tokenized_dataset.remove_columns(["text"])

# Set the dataset format to torch for PyTorch compatibility
tokenized_dataset.set_format("torch")


In [ ]:
# Step 6: Print the final tokenized dataset structure
print(tokenized_dataset)

In [ ]:
from transformers import RobertaForSequenceClassification

# Load the pre-trained RoBERTa model
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)


In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",          # Where to save the results
    evaluation_strategy="epoch",     # Evaluate at the end of each epoch
    learning_rate=2e-5,              # Learning rate
    per_device_train_batch_size=16,  # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    num_train_epochs=3,              # Number of training epochs
    weight_decay=0.01,               # Weight decay for regularization
)

In [ ]:
# Define the Trainer
trainer = Trainer(
    model=model,                     # The pre-trained model
    args=training_args,              # The training arguments
    train_dataset=tokenized_dataset["train"],  # The training dataset
    eval_dataset=tokenized_dataset["test"],    # The evaluation dataset
    tokenizer=tokenizer,             # The tokenizer
)

# Start training
trainer.train()
